In [1]:
USE ALM_TEST;
GO

-- Если представление уже существует, можно его переопределить:
CREATE OR ALTER VIEW [WORK].[vw_ProlongationAnalysis_OpenAndExit_BalanceRub]
AS
/*
  Это представление агрегирует данные по депозитам с 2024 года.
  Оно рассчитывает:
  1. Показатели по "открытым" депозитам – определяется по DT_OPEN:
     - Только депозиты, открытые в заданном месяце (согласно cteMonths) и прожившие ≥ 10 дней.
     - Для каждого депозита рассчитывается число пролонгаций (0/1/2/3+) через до 3 LEFT JOIN
       (предыдущие вклады учитываются только если они тоже прожили ≥ 10 дней).
     - Вычисляются агрегаты: количество сделок, сумма BALANCE_RUB, сумма пролонгаций (в рублях) и т.д.
  2. Показатели по "выходам" – депозиты, у которых фактическое закрытие (DT_CLOSE_FACT)
     произошло в данном месяце. Для них также берётся BALANCE_RUB и определяется сегментация
     (с использованием тех же полей: SEG_NAME, CUR и бакета срочности, определяемого по DaysLived).
  Затем данные агрегируются по четырём измерениям: последний день месяца, сегмент, валюта и бакет.
  Для объединения агрегатов по открытым сделкам и по выходам используется FULL OUTER JOIN по ключу
  (MonthEnd, SegmentGrouping, CurrencyGrouping, TermBucketGrouping).
*/
WITH
-----------------------------
-- 1) Генерация месяцев (MonthEnd)
-----------------------------
cteMonths AS (
    SELECT CONVERT(date, '2024-01-31') AS MonthEnd
    UNION ALL
    SELECT EOMONTH(DATEADD(MONTH, 1, MonthEnd))
    FROM cteMonths
    WHERE EOMONTH(DATEADD(MONTH, 1, MonthEnd)) <= '2025-02-28'
),
-----------------------------
-- 2) Открытые депозиты (по DT_OPEN) – только те, которые прожили ≥ 10 дней
-----------------------------
DealsInMonth AS (
    SELECT
         M.MonthEnd,
         CAST(dc.CON_ID AS BIGINT) AS CON_ID,
         dc.SEG_NAME,
         dc.CUR,
         dc.DT_OPEN,
         dc.BALANCE_RUB,
         DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) AS DaysLived
    FROM cteMonths M
    JOIN [LIQUIDITY].[liq].[DepositContract_all] dc
      ON dc.DT_OPEN >= DATEADD(DAY, 1, EOMONTH(M.MonthEnd, -1))  -- начало месяца
     AND dc.DT_OPEN <  DATEADD(DAY, 1, M.MonthEnd)               -- начало следующего месяца
     AND dc.CLI_SUBTYPE = 'INDIV'
     AND dc.PROD_NAME   != 'Эскроу'
     AND dc.DT_CLOSE_PLAN != '4444-01-01'
     AND DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) >= 10
),
-----------------------------
-- 3) Привязываем бакеты срочности (используя TERM_GROUP из man_TermGroup)
-----------------------------
DealsInMonthBucket AS (
    SELECT
         dm.MonthEnd,
         dm.CON_ID,
         dm.SEG_NAME,
         dm.CUR,
         dm.DT_OPEN,
         dm.BALANCE_RUB,
         dm.DaysLived,
         tg.TERM_GROUP AS TermBucket
    FROM DealsInMonth dm
    LEFT JOIN [ALM_TEST].[WORK].[man_TermGroup] tg
       ON dm.DaysLived >= tg.TERM_FROM
      AND dm.DaysLived <= tg.TERM_TO
),
-----------------------------
-- 4) Определяем число пролонгаций (до 3 звеньев) для открытых депозитов
-----------------------------
DealsWithProlong AS (
    SELECT
         dmb.MonthEnd,
         dmb.CON_ID,
         dmb.SEG_NAME,
         dmb.CUR,
         dmb.DT_OPEN,
         dmb.BALANCE_RUB,
         dmb.TermBucket,
         CASE
           WHEN p1.CON_ID IS NULL OR old1.CON_ID IS NULL THEN 0
           WHEN p2.CON_ID IS NULL OR old2.CON_ID IS NULL THEN 1
           WHEN p3.CON_ID IS NULL OR old3.CON_ID IS NULL THEN 2
           ELSE 3
         END AS ProlongCount
    FROM DealsInMonthBucket dmb
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p1
         ON p1.CON_ID = dmb.CON_ID
        AND p1.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old1
         ON old1.CON_ID = p1.CON_ID_REL
        AND DATEDIFF(DAY, old1.DT_OPEN, old1.DT_CLOSE) >= 10
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p2
         ON p2.CON_ID = old1.CON_ID
        AND p2.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old2
         ON old2.CON_ID = p2.CON_ID_REL
        AND DATEDIFF(DAY, old2.DT_OPEN, old2.DT_CLOSE) >= 10
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p3
         ON p3.CON_ID = old2.CON_ID
        AND p3.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old3
         ON old3.CON_ID = p3.CON_ID_REL
        AND DATEDIFF(DAY, old3.DT_OPEN, old3.DT_CLOSE) >= 10
),
-----------------------------
-- 5) Агрегируем открытые сделки по сегментации
-----------------------------
OpenAggregated AS (
    SELECT 
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         COUNT(*) AS OpenedDeals,
         SUM(BALANCE_RUB) AS Summ_BalanceRub,
         SUM(CASE WHEN ProlongCount > 0 THEN 1 ELSE 0 END) AS Count_Prolong,
         SUM(CASE WHEN ProlongCount = 1 THEN 1 ELSE 0 END) AS Count_1yProlong,
         SUM(CASE WHEN ProlongCount = 2 THEN 1 ELSE 0 END) AS Count_2yProlong,
         SUM(CASE WHEN ProlongCount >= 3 THEN 1 ELSE 0 END) AS Count_3plusProlong,
         SUM(CASE WHEN ProlongCount > 0 THEN BALANCE_RUB ELSE 0 END) AS Sum_ProlongRub
    FROM DealsWithProlong
    GROUP BY MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
),
-----------------------------
-- 6) Выходы депозитов (по DT_CLOSE_FACT) с сегментацией
-----------------------------
ExitsInMonthDetailed AS (
    SELECT
         M.MonthEnd,
         CAST(dc.CON_ID AS BIGINT) AS CON_ID,
         dc.SEG_NAME,
         dc.CUR,
         dc.DT_CLOSE_FACT,
         DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) AS DaysLived,
         dc.BALANCE_RUB
    FROM cteMonths M
    JOIN [LIQUIDITY].[liq].[DepositContract_all] dc
         ON dc.DT_CLOSE_FACT >= DATEADD(DAY, 1, EOMONTH(M.MonthEnd, -1))
         AND dc.DT_CLOSE_FACT < DATEADD(DAY, 1, M.MonthEnd)
         AND dc.CLI_SUBTYPE = 'INDIV'
         AND dc.PROD_NAME != 'Эскроу'
         AND dc.DT_CLOSE_PLAN != '4444-01-01'
),
ExitsInMonthBucket AS (
    SELECT
         eid.MonthEnd,
         eid.CON_ID,
         eid.SEG_NAME,
         eid.CUR,
         eid.DT_CLOSE_FACT,
         eid.DaysLived,
         eid.BALANCE_RUB,
         tg.TERM_GROUP AS TermBucket
    FROM ExitsInMonthDetailed eid
    LEFT JOIN [ALM_TEST].[WORK].[man_TermGroup] tg
         ON eid.DaysLived >= tg.TERM_FROM
         AND eid.DaysLived <= tg.TERM_TO
),
ExitsAggregated AS (
    SELECT
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         COUNT(*) AS ExitCount,
         SUM(BALANCE_RUB) AS Summ_Exits
    FROM ExitsInMonthBucket
    GROUP BY MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
)
-----------------------------
-- 7) Объединяем агрегаты по открытым сделкам и по выходам
--    FULL OUTER JOIN по ключу (MonthEnd, SegmentGrouping, CurrencyGrouping, TermBucketGrouping)
-----------------------------
SELECT
    COALESCE(o.MonthEnd, e.MonthEnd) AS MonthEnd,
    CASE WHEN GROUPING(COALESCE(o.SegmentGrouping, e.SegmentGrouping)) = 1
         THEN N'Все сегменты'
         ELSE COALESCE(o.SegmentGrouping, e.SegmentGrouping)
    END AS SegmentGrouping,
    CASE WHEN GROUPING(COALESCE(o.CurrencyGrouping, e.CurrencyGrouping)) = 1
         THEN N'Все валюты'
         ELSE COALESCE(o.CurrencyGrouping, e.CurrencyGrouping)
    END AS CurrencyGrouping,
    CASE WHEN GROUPING(COALESCE(o.TermBucketGrouping, e.TermBucketGrouping)) = 1
         THEN N'Все бакеты'
         ELSE COALESCE(o.TermBucketGrouping, e.TermBucketGrouping)
    END AS TermBucketGrouping,
    ISNULL(o.OpenedDeals,0) AS OpenedDeals,
    ISNULL(o.Summ_BalanceRub,0) AS Summ_BalanceRub,
    ISNULL(o.Count_Prolong,0) AS Count_Prolong,
    ISNULL(o.Count_1yProlong,0) AS Count_1yProlong,
    ISNULL(o.Count_2yProlong,0) AS Count_2yProlong,
    ISNULL(o.Count_3plusProlong,0) AS Count_3plusProlong,
    CASE WHEN ISNULL(o.Summ_BalanceRub,0) > 0 THEN
         1.0 * o.Sum_ProlongRub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_пролонгаций_Rub],
    ISNULL(e.ExitCount,0) AS ExitCount,
    ISNULL(e.Summ_Exits,0) AS Summ_Exits,
    CASE WHEN ISNULL(o.Summ_BalanceRub,0) > 0 THEN
         1.0 * e.Summ_Exits / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_выходов_Rub]
FROM OpenAggregated o
FULL OUTER JOIN ExitsAggregated e
    ON o.MonthEnd = e.MonthEnd
   AND o.SegmentGrouping = e.SegmentGrouping
   AND o.CurrencyGrouping = e.CurrencyGrouping
   AND o.TermBucketGrouping = e.TermBucketGrouping
ORDER BY
    COALESCE(o.MonthEnd, e.MonthEnd);
GO


SyntaxError: invalid character '–' (U+2013) (3093399885.py, line 10)